In [69]:
import requests
import pandas
import base64
from datetime import date,datetime,timedelta
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
import json
import configs
from SpotifyAPI import SpotifyAPI

In [2]:
#Connect to MySQL DB

connection = mysql.connector.connect(host = "localhost", database = "jason", user = "root", password = configs.password)

In [3]:
client_id = configs.client_id
client_secret = configs.client_secret

client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())  #To change this string to base64 encoded string

In [4]:
# do a lookup for a token, this token is for future requests
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = { 
        "grant_type" : "client_credentials"
}
token_headers = {
        "Authorization" : f"Basic {client_creds_b64.decode()}"
}

In [5]:
#Call object and perform authorization
spotify = SpotifyAPI(client_id, client_secret)

spotify.perform_auth()
print(spotify.perform_auth())
access_token = spotify.access_token

Success


In [21]:
#Use Access Token to access Spotify API
print(spotify.access_token_expires)


header = {
    "Authorization" : f"Bearer {access_token}"
}


#endpoint for tracks has to look like this "https://api.spotify.com/v1/search?q=tania%20bowra&type=artist"
#encode spaces with "%20" or "+"
#endpoint = f"https://api.spotify.com/v1/search?q={data['q'].replace(' ','%20').lower()}&type={data['type']}&limit={data['limit']}"

endpoint = "https://api.spotify.com/v1/playlists/37i9dQZEVXbMDoHDwVN2tF"

print(endpoint)

2020-08-03 16:33:56.519668
https://api.spotify.com/v1/playlists/37i9dQZEVXbMDoHDwVN2tF


In [22]:
r = requests.get(endpoint, headers = header)
print(r.status_code)

200


In [72]:
data = r.json()
playlist_json = data['tracks']
#print(data['tracks'].keys())
print(data['tracks']['items'][0]['track'].keys())

playlist_data = []

for i in range(playlist_json['total']):
    

    artist_name = ''
    for j in range(len(playlist_json['items'][i]['track']['artists'])):
        artist_name = artist_name + ',' + data['tracks']['items'][i]['track']['artists'][j]['name']
    
    track_tuple = (artist_name[1:],
                   playlist_json['items'][i]['track']['id'],
                   playlist_json['items'][i]['track']['name'],
                   playlist_json['items'][i]['track']['duration_ms'],
                   playlist_json['items'][i]['track']['explicit'],
                   playlist_json['items'][i]['track']['popularity'],
                   playlist_json['items'][i]['track']['album']['id'],
                   playlist_json['items'][i]['track']['album']['name'],
                   playlist_json['items'][i]['track']['album']['album_type'],
                   playlist_json['items'][i]['track']['album']['release_date'],
                   datetime.today().strftime('%Y-%m-%d')
                    )
    
    
    playlist_data.append(track_tuple)
    
#print(data['tracks']['items'][0]['track']['artists'][0])
print(playlist_data[0])

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
('Jawsh 685,Jason Derulo', '1xQ6trAsedVPCdbtDAmk0c', 'Savage Love (Laxed - Siren Beat)', 171374, True, 96, '1XMw3pBrYeXzNXZXc84DNw', 'Savage Love (Laxed - Siren Beat)', 'single', '2020-06-11', '2020-08-03')


In [73]:
#Insert data into MySQL 

insert_query = """INSERT INTO spotify_daily_top_50_global (artist_name, track_id, track_name, duration_ms, explicit, popularity, album_id, album_name, album_type, release_date, dt)    
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                """
insert_data = playlist_data

try: 
    cursor = connection.cursor()
    cursor.executemany(insert_query,insert_data)
    connection.commit()
    print(cursor.rowcount,"Record inserted successfully into spotify_daily_top_50_global table")
except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

50 Record inserted successfully into spotify_daily_top_50_global table
MySQL connection is closed
